<a href="https://colab.research.google.com/github/sarang-manohar/hogwarts/blob/main/academia/sig788/code/task_4_1_P_Sarang_Manohar(s223504903)_Part2_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# Install necessary libraries
!pip install fiftyone
!pip install azure-cognitiveservices-vision-customvision
!pip install modin
!pip install jupyter-server-proxy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [91]:
import json

with open('creds.txt') as f:
    contents = f.read()
    creds = json.loads(contents)

In [92]:
credentials = ApiKeyCredentials(in_headers={"Training-key": creds['training_key']})
trainer = CustomVisionTrainingClient(creds['ENDPOINT'], credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": creds['prediction_key']})
predictor = CustomVisionPredictionClient(creds['ENDPOINT'], prediction_credentials)

In [94]:
with open('project_id.txt') as f:
    project_id = f.read()
project_id

'e9c32e6a-a241-401c-bc7f-e859fb047664'

In [95]:
tag = trainer.create_tag(project_id, "bowl")

In [113]:
# Import fiftyone
import fiftyone.zoo as fo

# Import data dataset 
def load_dataset(max_samples, seed):
    dataset_train = fo.load_zoo_dataset(
        "coco-2017",
        split="train",
        label_types=["detections"],
        classes=["bowl"],
        max_samples=max_samples,
        seed=seed,
        dataset_name="oi-bowl-train" + str(seed),
    )
    return dataset_train

In [114]:
dataset_train = load_dataset(max_samples=1000, seed=2)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'
2023-04-03 09:31:19,670 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-04-03 09:31:21,089 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-04-03 09:31:22,830 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-04-03 09:31:25,373 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-04-03 09:31:29,192 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-04-03 09:31:33,786 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2023-04-03 09:31:38,063 - distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (thre

999 images found; downloading the remaining 1


INFO:fiftyone.utils.coco:999 images found; downloading the remaining 1


...

INFO:fiftyone.utils.coco:Writing annotations for 1000 downloaded samples to '/root/fiftyone/coco-2017/train/labels.json'


Dataset info written to '/root/fiftyone/coco-2017/info.json'


INFO:fiftyone.zoo.datasets:Dataset info written to '/root/fiftyone/coco-2017/info.json'


Loading existing dataset 'oi-bowl-train2'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'oi-bowl-train2'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [115]:
with open('/root/fiftyone/coco-2017/train/labels.json') as f:
    contents = f.read()
    annotations = json.loads(contents)
    
import modin.pandas as pd

tmp = pd.DataFrame(annotations['images'])
tmp = tmp[['file_name','height','width','id']].copy()
tmp2 = pd.DataFrame(annotations['annotations'])
tmp2 = tmp2[['image_id','bbox','area','category_id']].copy()
tmp2.rename(columns={'image_id':'id'}, inplace=True)

#merge dataframes
merged_data = pd.merge(tmp,tmp2, how='inner')
tmp3 = pd.DataFrame(annotations['categories'])
tmp3 = tmp3[tmp3['name']=='bowl'].copy()
tmp3.rename(columns={'id':'category_id'}, inplace=True)
merged_data = pd.merge(merged_data,tmp3, how='inner')
merged_data[['Left','Top','Width','Height']] = pd.DataFrame(merged_data.bbox.tolist(), index= merged_data.index)
merged_data['Left'] = merged_data['Left'] / merged_data['width'] 
merged_data['Top'] = merged_data['Top'] / merged_data['height'] 
merged_data['Width'] = merged_data['Width'] / merged_data['width'] 
merged_data['Height'] = merged_data['Height'] / merged_data['height'] 
data_to_upload = merged_data[['file_name','Left','Top','Width','Height']] 
dict = data_to_upload.set_index('file_name').T.to_dict('list')

In [116]:
import shutil

data_dir = os.path.join('/root/fiftyone/coco-2017/train/data')
temp_dir = os.path.join('/root/fiftyone/coco-2017/train/temp')
if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)

data = pd.DataFrame(dict)

In [129]:
batch_size = 50
for i in range(0, data.shape[1], batch_size):
    if i != 0:
            print(i," files uploaded so far")
    # Get the next batch of files
    batch_df = data.iloc[:,i:i+batch_size]
    tagged_images_with_regions = []
    file_list = []

    for file_name in batch_df.to_dict().keys():
        x,y,w,h = batch_df[file_name]
        regions = [ Region(tag_id=tag.id, left=x,top=y,width=w,height=h) ]
        file_list.append(file_name)

    for file in file_list:
        shutil.copy(os.path.join(data_dir, file), os.path.join(temp_dir, file))
        with open(os.path.join (temp_dir, file ), mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file, contents=image_contents.read(), regions=regions))

        upload_result = trainer.create_images_from_files(project_id, ImageFileCreateBatch(images=tagged_images_with_regions))
        if not upload_result.is_batch_successful:
            print("Image batch upload failed.")
            for image in upload_result.images:
                print("Image status: ", image.status)
            exit(-1)
        for file in file_list:
            os.remove(os.path.join(temp_dir, file))
        


INFO:distributed.scheduler:User asked for computation on lost data, apply_func-fd020d6e-4f7a-47f3-ad2b-4b79de10e785
INFO:distributed.scheduler:User asked for computation on lost data, lambda-5d4efd5a121eeba5f961b8be5c8fb549
INFO:distributed.scheduler:User asked for computation on lost data, lambda-78057c642446d62587baaf4d1320a15c


CancelledError: ignored